### 수업 운영

- 09:00 ~ 12:00 프로젝트 수행(쉬는시간 자유롭게)
- 12:00 ~ 13:00 점심시간
- 13:00 ~ 14:00 프로젝트 수행(쉬는시간 자유롭게)
- 14:00 ~ 14:10 쉬는시간
- 14:10 ~ 17:00 (평일 저녁에 했던 것 처럼 수업 진행)

# 오늘의 프로젝트: 보스턴 집값 예측을 위한 모듈 제작

- boston_loader.py : 보스턴 집값 데이터를 로드하는 클래스를 가진 모듈
- boston_preprocessor: 보스턴 집값 데이터를 전처리하는 클래스를 가진 모듈
  - 결측치가 있는지 확인하고 결측치가 있다면 결측치를 처리합니다.
  - standardscaler를 적용하여 y를 제외한 X 값을 정규화합니다
  - 학습을 위해 데이터를 X_train, y_train, X_test, y_test로 split 합니다.
- DT.py, RF.py, XGB.py: 디시젼트리, 랜덤포레스트, XGBoost 모델을 담은 클래스를 제작합니다.
  - 집값 데이터의 형태에 맞춰 분류, 회귀 중 하나의 유형에 맞춰 클래스를 제작합니다.
  - 학습, 예측 값 추출을 진행하고 평가지표를 산출합니다.

**참고사항**

- 이번 프로젝트의 정답은 이미 실행해보았던 `data_lodaers폴더`, `models 폴더`, `preprocessor 폴더`, `05 XGB.ipynb` 이나 기능 구현은 사람마다 다른 코드로 구현하게 됩니다. 그러니 여러분만의 코드를 사용하여 결론적으로만 기 실습했던 기능들과 여러분의 기능이 같도록 구현되면 됩니다.
- 모듈(py 파일)을 생성하시어 구현하셔도 좋고 노트북 파일 내에 클래스만 선언하시어 진행해보셔도 좋습니다.
  - 권장 드리는 것은 노트북 파일 내에 클래스를 먼저 선언하여 코드가 정상적으로 동작하는지 확인 후 모듈화 하시는 것입니다.
- 클래스 구현시 오버라이딩을 활용하여 추가적인 모듈을 생성하여 활용하여도 좋습니다.

In [2]:
# prompt: data_loaders에 있는 클래스를 불러올 수 있도록 sys 를 사용하여 경로를 추가해줘

import sys
sys.path.append('/content/data_loaders') # data_loaders 폴더의 경로를 추가합니다.  data_loaders 폴더가 현재 작업 디렉토리에 있어야 합니다.

from boston_loader import BostonLoader

ModuleNotFoundError: No module named 'base_data_loader'

In [3]:
"boston.csv".split(".")[1]

'csv'

In [ ]:
# prompt: boston.csv 각 컬럼의 의미를 아주 상세하게 주석으로 작성해.
# 컬럼의 리스트 = [CRIM	ZN	INDUS	CHAS	NOX	RM	AGE	DIS	RAD	TAX	PTRATIO	B	LSTAT	PRICE]


# CRIM: 자치시(town)별 1인당 범죄율
# ZN: 25,000 평방피트를 초과하는 거주지역의 비율
# INDUS: 비소매상업지역이 점유하고 있는 토지의 비율
# CHAS: 찰스강에 대한 더미변수(강의 경계에 위치한 경우는 1, 아니면 0)
# NOX: 10ppm 당 농축 일산화질소
# RM: 주택 1가구당 평균 방의 개수
# AGE: 1940년 이전에 건축된 소유주택의 비율
# DIS: 5개의 보스턴 직업센터까지의 가중 거리
# RAD: 방사형 고속도로 접근성 지수
# TAX: 10,000 달러 당 재산세율
# PTRATIO: 자치시(town)별 학생/교사 비율
# B: 1000(Bk - 0.63)^2, 여기서 Bk는 자치시별 흑인의 비율을 말한다.
# LSTAT: 모집단의 하위 계층의 비율(%)
# PRICE: 본인 소유의 주택 가격(중앙값, $1,000)

In [ ]:
나는 모듈화 + 상속을 이용한 클래스 구현 까지 다 했다 = 0
모듈화까지는 완료 했다 1
노트북 안에서 클래스는 다 구현 했다 2
노트북 안에서 클래스를 다 구현 하면서 상속도 함께 적용하여 구현 했다 3
함수로만 구현을 했다 4
아예 풀지 못했다 5

In [1]:
import pandas as pd
df = pd.read_csv("boston.csv")
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
